In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd
import numpy as np
import seaborn as sns

from itertools import compress


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectPercentile, mutual_info_classif
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB

from catboost import CatBoostClassifier
from xgboost import XGBRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from time import time

In [4]:
# import data from files
X_in = pd.read_csv('../Data/train_values.csv')
Y_in = pd.read_csv('../Data/train_labels.csv')


# score set
X_score = pd.read_csv('../Data/test_values.csv')

In [5]:
data = pd.DataFrame(X_in)
data = data.merge(Y_in, on='row_id')

In [6]:
# lender
# 6111 distinct values
lenders_data = data[["lender", "accepted"]].groupby(by="lender").sum() / data[["lender", "accepted"]].groupby(by="lender").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal')

lenders_binned = discretizer.fit_transform(lenders_data)
lenders_data["lender_bin"] = lenders_binned

lenders_dict = lenders_data["lender_bin"].to_dict()

data["lender"].replace(lenders_dict, inplace=True)
X_score["lender"].replace(lenders_dict, inplace=True)

C:\Users\Markus.Haftstein.ADASTRACORPNET\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


In [7]:
# county_code
# 318 distinct values
county_data = data[["county_code", "accepted"]].groupby(by="county_code").sum() / data[["county_code", "accepted"]].groupby(by="county_code").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal')

county_binned = discretizer.fit_transform(county_data)
county_data["county_bin"] = county_binned

county_dict = county_data["county_bin"].to_dict()

data["county_code"].replace(county_dict, inplace=True)
X_score["county_code"].replace(county_dict, inplace=True)

In [8]:
# state_code
# 53 distinct values
state_data = data[["state_code", "accepted"]].groupby(by="state_code").sum() / data[["state_code", "accepted"]].groupby(by="state_code").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal')

state_binned = discretizer.fit_transform(state_data)
state_data["state_bin"] = state_binned

state_dict = state_data["state_bin"].to_dict()

data["state_code"].replace(state_dict, inplace=True)
X_score["state_code"].replace(state_dict, inplace=True)

In [9]:
# msa_md
# 409 distinct values
msa_md_data = data[["msa_md", "accepted"]].groupby(by="msa_md").sum() / data[["msa_md", "accepted"]].groupby(by="msa_md").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=6, encode='ordinal')

msa_md_binned = discretizer.fit_transform(msa_md_data)
msa_md_data["msa_md_bin"] = msa_md_binned

msa_md_dict = msa_md_data["msa_md_bin"].to_dict()

data["msa_md"].replace(msa_md_dict, inplace=True)
X_score["msa_md"].replace(msa_md_dict, inplace=True)

In [10]:
# loan_purpose
# 3 distinct values
lp_data = data[["loan_purpose", "accepted"]].groupby(by="loan_purpose").sum() / data[["loan_purpose", "accepted"]].groupby(by="loan_purpose").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal')

lp_binned = discretizer.fit_transform(lp_data)
lp_data["loan_purpose_bin"] = lp_binned

lp_dict = lp_data["loan_purpose_bin"].to_dict()

data["loan_purpose"].replace(lp_dict, inplace=True)
X_score["loan_purpose"].replace(lp_dict, inplace=True)

In [11]:
# applicant_race
# 7 distinct values
applicant_race_data = data[["applicant_race", "accepted"]].groupby(by="applicant_race").sum() / data[["applicant_race", "accepted"]].groupby(by="applicant_race").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=7, encode='ordinal')

applicant_race_binned = discretizer.fit_transform(applicant_race_data)
applicant_race_data["applicant_race_bin"] = applicant_race_binned

applicant_race_dict = applicant_race_data["applicant_race_bin"].to_dict()

data["applicant_race"].replace(applicant_race_dict, inplace=True)
X_score["applicant_race"].replace(applicant_race_dict, inplace=True)

In [12]:
# property_type
# 3 distinct values
property_type_data = data[["property_type", "accepted"]].groupby(by="property_type").sum() / data[["property_type", "accepted"]].groupby(by="property_type").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal')

property_type_binned = discretizer.fit_transform(property_type_data)
property_type_data["property_type_bin"] = property_type_binned

property_type_dict = property_type_data["property_type_bin"].to_dict()

data["property_type"].replace(property_type_dict, inplace=True)
X_score["property_type"].replace(property_type_dict, inplace=True)

In [13]:
# applicant_ethnicity
# 4 distinct values
applicant_ethnicity_data = data[["applicant_ethnicity", "accepted"]].groupby(by="applicant_ethnicity").sum() / data[["applicant_ethnicity", "accepted"]].groupby(by="applicant_ethnicity").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=4, encode='ordinal')

applicant_ethnicity_binned = discretizer.fit_transform(applicant_ethnicity_data)
applicant_ethnicity_data["applicant_ethnicity_bin"] = applicant_ethnicity_binned

applicant_ethnicity_dict = applicant_ethnicity_data["applicant_ethnicity_bin"].to_dict()

data["applicant_ethnicity"].replace(applicant_ethnicity_dict, inplace=True)
X_score["applicant_ethnicity"].replace(applicant_ethnicity_dict, inplace=True)

In [14]:
# loan_type
# 4 distinct values
loan_type_data = data[["loan_type", "accepted"]].groupby(by="loan_type").sum() / data[["loan_type", "accepted"]].groupby(by="loan_type").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=4, encode='ordinal')

loan_type_binned = discretizer.fit_transform(loan_type_data)
loan_type_data["loan_type_bin"] = loan_type_binned

loan_type_dict = loan_type_data["loan_type_bin"].to_dict()

data["loan_type"].replace(loan_type_dict, inplace=True)
X_score["loan_type"].replace(loan_type_dict, inplace=True)

In [15]:
# occupancy
# 3 distinct values
occupancy_data = data[["occupancy", "accepted"]].groupby(by="occupancy").sum() / data[["occupancy", "accepted"]].groupby(by="occupancy").count()

from sklearn.preprocessing import KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal')

occupancy_binned = discretizer.fit_transform(occupancy_data)
occupancy_data["occupancy_bin"] = occupancy_binned

occupancy_dict = occupancy_data["occupancy_bin"].to_dict()

data["occupancy"].replace(occupancy_dict, inplace=True)
X_score["occupancy"].replace(occupancy_dict, inplace=True)

In [16]:
data["LoanIncomeRatio"]=data["loan_amount"]/data["applicant_income"]
X_score["LoanIncomeRatio"]=X_score["loan_amount"]/X_score["applicant_income"]

In [17]:
data.drop("accepted", axis=1, inplace=True)

In [18]:
data.to_csv("X_binned.csv")
X_score.to_csv("X_score_binned.csv")